In [6]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time

import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [7]:
from config import key_id, secret_key

In [8]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [9]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [10]:
account = api.get_account()
account

Account({   'account_blocked': False,
    'account_number': '010320JEWA',
    'buying_power': '611.4474',
    'cash': '-2206.5963',
    'created_at': '2020-03-20T17:46:19.948568Z',
    'currency': 'USD',
    'daytrade_count': 2,
    'daytrading_buying_power': '0',
    'equity': '2818.0437',
    'id': '176ffd14-7e0e-4957-9ad5-98306425fa04',
    'initial_margin': '2512.32',
    'last_equity': '2727.4',
    'last_maintenance_margin': '1611.04',
    'long_market_value': '5024.64',
    'maintenance_margin': '1507.392',
    'multiplier': '2',
    'pattern_day_trader': False,
    'portfolio_value': '2818.0437',
    'regt_buying_power': '611.4474',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '613.44',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [11]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-07'

In [12]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=5, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=2, price='low')
        
        dataframe['sema_high'] = tab.EMA(dataframe, timeperiod=6, price='high')
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['srsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        #srsi used for finding buys at 71
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        
        
        dataframe["upperband"], dataframe["middleband"], dataframe["lowerband"] = ta.BBANDS(dataframe['close'], timeperiod=10, nbdevup=1, nbdevdn=1)
        
        # required for graphing

        return dataframe

In [13]:
len(api.list_orders(status='closed'))

50

In [14]:
account = api.get_account()
print(account.status)


ACTIVE


In [15]:
#list of symbols to look at 
# DRI CLX 

In [20]:
symbol ='AMD'
returned_data =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-27', to='2020-05-04').df
#api.get_barset(symbol,barTimeframe,limit=150).df


# Calculated trading indicators
data=populateindicators(returned_data)

In [21]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [22]:
data

,open,high,low,close,volume,ema_high,ema_close,ema_low,sema_high,mfi,ultosc,minusdi,plusdi,srsi,rsi,adx,upperband,middleband,lowerband
timestamp,,,,,,,,,,,,,,,,,,,
2020-04-27 04:01:00-04:00,57.0000,57.03,57.0000,57.03,770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-27 04:03:00-04:00,57.1100,57.12,57.1100,57.12,1494.0,NaN,NaN,57.055000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-27 04:04:00-04:00,57.1900,57.19,57.1900,57.19,461.0,NaN,NaN,57.145000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-27 04:06:00-04:00,57.2000,57.20,57.2000,57.20,614.0,NaN,57.135000,57.181667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-27 04:07:00-04:00,57.2100,57.21,57.2100,57.21,168.0,57.150000,57.165000,57.200556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-04 19:55:00-04:00,52.6800,52.69,52.6799,52.68,4093.0,52.666591,52.662340,52.671993,52.663634,68.835323,61.076317,24.276925,38.878398,62.128099,64.111277,8.812927,52.662656,52.641,52.619344
2020-05-04 19:56:00-04:00,52.6799,52.68,52.6799,52.68,2697.0,52.671060,52.669404,52.677264,52.668310,59.906786,55.063412,24.266873,38.862301,62.128099,64.111277,9.834857,52.670166,52.646,52.621834
2020-05-04 19:57:00-04:00,52.6800,52.70,52.6800,52.70,3309.0,52.680707,52.681642,52.679088,52.677364,65.412505,72.146743,22.279940,43.868158,64.907849,67.815320,11.463524,52.681276,52.654,52.626724


In [23]:
# Use Dtale for graphs and inspection and find where the indicators indicate growth
import dtale

df=data
dtale.show(df, ignore_duplicate=True)


Executing shutdown due to inactivity...


2020-05-07 08:50:30,902 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2020-05-07 08:50:38,919 - INFO     - Executing shutdown...


In [15]:
#

In [25]:
#list of symbols to look at 
# DRI CLX